In [90]:
FTRAIN = 'data/training.csv' #getting training data

In [98]:
from lasagne import layers
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import BatchIterator
import theano

In [99]:
height, width, deph = 28,28,1 #параметры изображение 28x28 в сером спектре
batch_size = 64 # рассматриваем 128 обучающий парметров на каждой итерации
num_epochs = 1 # итерируемся 20 раз по обучающей выборке 
hidden_size = 128 # по 512 нейронов в каждом (из двух) слоев НС
num_classes = 30 # 10 классов 
num_train = 60000 # 60к тренеровочных МНИСТ примеров
num_test = 10000 #10к тестовых МНИСТ примеров

In [100]:
import os
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from keras.utils import np_utils # утилита для декодирования истенных значений (перевод в Унитарный/Двоичный код)

def load(test=False, cols=None): #loads data from FTRAIN if true, otherwise from FTRAIN
  
    fname = FTEST if test else FTRAIN
    X = read_csv(fname, sep=' ') 
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    #print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255
    #X = X.reshape(0, 784)
    X = X.astype(np.float32)

   # if not test:  # only FTRAIN has any target columns
    y = df[df.columns[:-1]].values
    y = (y - 48) / 48  # scale target coordinates to [-1, 1]
    X, y = shuffle(X, y, random_state=42)  # shuffle train data
    y = y.astype(np.float32) 
    #else:
      #  y = None
        
    #y = np_utils.to_categorical(y, num_classes) #кодирование меток в двоичный код

    return X, y


X, y = load()
#print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
#print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))


In [101]:
from keras.layers import Input, Dense # 2 типа слоев НС
from keras.models import Model  #класс для определения и обучений НС

inp = Input(shape=(9216,)) #на входе вектор размера 784
hidden_1 = Dense(hidden_size, activation ='relu')(inp) #первый скрытый ReLu слой
hidden_2 = Dense(hidden_size, activation = 'relu')(hidden_1) #второй скрытый ReLu слой
out = Dense(num_classes, activation='softmax')(hidden_2) # выходной softmax слой
# softmax превращает вектор действительных чисел в вектор вероятностей

model = Model(input = inp, output = out)

In [102]:
model.compile(loss='categorical_crossentropy', #функция потерь крос - энтропии
             optimizer = 'adam', #отпимизатор Адама
              metrics = ['accuracy'] #точность - доля входных данных, отнесенных к правильному классу
             )

In [103]:
model.fit(X, y, batch_size = batch_size, nb_epoch = num_epochs)
#model.evaluate(X, y,verbose=1) #оценка модели на тестовом наборе 

Epoch 1/1
2140/2140 [==============================] - 0s - loss: -63.4743 - acc: 0.1636     